In [151]:
from dotenv import load_dotenv
from langchain_community.utilities import SQLDatabase
from langchain_groq import ChatGroq
from langchain_community.chat_models import ChatOllama
import os
load_dotenv()

True

# Connect To Database

In [152]:
user = os.getenv("DB_USER")
password = os.getenv("DB_PASSWORD")
host = os.getenv("DB_HOST")
port = os.getenv("DB_PORT")
database = os.getenv("DB_NAME")
db_url=f"mysql+mysqlconnector://{user}:{password}@{host}:{port}/{database}"
db = SQLDatabase.from_uri(db_url)
print(db.dialect)
print(f"Get All Tables:{db.get_usable_table_names()}")

def get_schema(_):
    return db.get_table_info()



mysql
Get All Tables:['auth_group', 'auth_group_permissions', 'auth_permission', 'django_admin_log', 'django_celery_beat_clockedschedule', 'django_celery_beat_crontabschedule', 'django_celery_beat_intervalschedule', 'django_celery_beat_periodictask', 'django_celery_beat_periodictasks', 'django_celery_beat_solarschedule', 'django_content_type', 'django_cron_cronjoblock', 'django_cron_cronjoblog', 'django_migrations', 'django_session', 'farmers_cerealpreferencecompletion', 'farmers_cerealspop', 'farmers_cerealspop_fk_product', 'farmers_cerealstagecompletion', 'farmers_commentreply', 'farmers_communitypost', 'farmers_cropimages', 'farmers_cropmapper', 'farmers_cropmaster', 'farmers_cropvariety', 'farmers_currentnews', 'farmers_disease_images_master', 'farmers_disease_images_master_fk_disease', 'farmers_diseasemaster', 'farmers_diseaseproductinfo', 'farmers_diseaseproductinfo_fk_product', 'farmers_diseasetranslation', 'farmers_diseasevideo', 'farmers_districtmaster', 'farmers_farmerlandadd

# Get all DatabaseNames

In [153]:
import mysql.connector
from mysql.connector import Error
def get_database_names(host: str, user: str, password: str, port: str):
    try:
        # Establish connection to the MySQL server
        connection = mysql.connector.connect(
            host=host,
            user=user,
            password=password,
            port=port
        )
        
        if connection.is_connected():
            cursor = connection.cursor()
            cursor.execute("SHOW DATABASES")  
            
            # Fetch all database names
            databases = cursor.fetchall()
            
            print("Available Databases:")
            for db in databases:
                print(db[0])  
                
            return databases
            
    except Error as e:
        print(f"Error: {str(e)}")
    finally:
        if connection.is_connected():
            cursor.close()
            connection.close()
            
get_database_names(host, user, password, port)

Available Databases:
agrisarathi
information_schema
mysql
performance_schema
sys


[('agrisarathi',),
 ('information_schema',),
 ('mysql',),
 ('performance_schema',),
 ('sys',)]

# Run Query

In [113]:
db.run("SELECT * FROM farmers_statemaster;")

"[(4, 'ANDHRA PRADESH', datetime.datetime(2024, 9, 4, 6, 55, 26, 170360), datetime.datetime(2024, 9, 4, 6, 55, 26, 170415), 0, 'आंध्र प्रदेश'), (5, 'ARUNACHAL PRADESH', datetime.datetime(2024, 9, 4, 6, 55, 26, 175800), datetime.datetime(2024, 9, 4, 6, 55, 26, 175826), 0, 'अरुणाचल प्रदेश'), (6, 'ASSAM', datetime.datetime(2024, 9, 4, 6, 55, 26, 178301), datetime.datetime(2024, 9, 4, 6, 55, 26, 178327), 0, 'असम'), (7, 'BIHAR', datetime.datetime(2024, 9, 4, 6, 55, 26, 180307), datetime.datetime(2024, 9, 4, 6, 55, 26, 180336), 0, 'बिहार'), (8, 'CHANDIGARH', datetime.datetime(2024, 9, 4, 6, 55, 26, 185713), datetime.datetime(2024, 9, 4, 6, 55, 26, 185757), 0, 'चंडीगढ़'), (9, 'CHHATTISGARH', datetime.datetime(2024, 9, 4, 6, 55, 26, 189291), datetime.datetime(2024, 9, 4, 6, 55, 26, 189322), 0, 'छत्तीसगढ'), (10, 'DADRA & NAGAR HAVELI', datetime.datetime(2024, 9, 4, 6, 55, 26, 191649), datetime.datetime(2024, 9, 4, 6, 55, 26, 191676), 0, 'दादरा एवं नगर हवेली'), (11, 'DAMAN AND DIU', datetime.dat

# Query Processing (To Get Sql Query)

In [150]:
from langchain_community.agent_toolkits.sql.toolkit import SQLDatabaseToolkit
from langchain_community.chat_models import ChatOllama
from langchain.chains import create_sql_query_chain
ollama_llm="llama3.1:8b"
llm = ChatOllama(model=ollama_llm)
agent_executor = create_sql_query_chain(llm, db) # Uses defalut langchain prompt
res=agent_executor.invoke({"question":"Give me list of farmers_profile that are part of fpo in fponsupplier_fpo"})
res

'SELECT id, first_name, last_name FROM fpoonsupplier_fpo WHERE is_part_of_fpo = True AND farmer_id IN ( SELECT id FROM farmers_profile )'

# Execute Sql Query

In [147]:
from langchain_community.tools.sql_database.tool import QuerySQLDataBaseTool
execute_query = QuerySQLDataBaseTool(db=db,llm=llm)
execute_query.invoke(res)

"Error: (mysql.connector.errors.ProgrammingError) 1146 (42S02): Table 'agrisarathi.farmers_profile' doesn't exist\n[SQL: SELECT * FROM farmers_profile WHERE mobile = '6003093159']\n(Background on this error at: https://sqlalche.me/e/20/f405)"

In [148]:
from operator import itemgetter
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough

answer_prompt = PromptTemplate.from_template(
     """Given the following user question, corresponding SQL query, and SQL result, answer the user question.

 Question: {question}
 SQL Query: {query}
 SQL Result: {result}
 Answer: """
 )

rephrase_answer = answer_prompt | llm | StrOutputParser() ## Prompt goes to llm i.e AI meesage then that message 

chain = (
     RunnablePassthrough.assign(query=agent_executor).assign(
         result=itemgetter("query") | execute_query
     )
     | rephrase_answer
 )

chain.invoke({"question": "Give me of all the details of user with mobile 6003093159  in farmers_profile"})


'Based on the SQL result, it appears that there is only one user with mobile number "6003093159" in the farmers_profile. \n\nHere are the details of this user:\n\n* Unique ID (78)\n* Auth Token (!RqMeKugEZjKFfKMZr9IUGSMomeCnjlwdybqIwxpk)\n* Admin status: No (None)\n* Profile picture URL: None\n* Mobile number: 6003093159\n* User type: Farmer\n* Has profile picture: Yes (1), but no image provided.\n* Is online: No (0)\n* Is active: No (0)\n* Has email verified: No (0)\n* Last seen datetime: September 10, 2024, at 12:26:52 PM\n* Last online datetime: September 10, 2024, at 12:27:47 PM\n* Number of notifications: 0\n\nIt seems that this user has not verified their email address and is currently offline.'